# Learning from Voxels

In [ ]:
import numpy as np
from PIL import Image
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

## The point cloud

In [ ]:
# Load point cloud data
pcd = o3d.io.read_point_cloud("../data/bun_zipper_res2.ply")

points = np.asarray(pcd.points)
x = points[:, 0]
y = points[:, 1]
z = points[:, 2]

point_cloud = go.Scatter3d(x=x,
                           y=y,
                           z=z,
                           mode='markers',
                           hovertemplate="<b>Point</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                           hoverlabel=dict(bgcolor='white'),
                           marker=dict(size=5,
                                       color=z,
                                       colorscale='magma'))

## Point clouds as voxel grids

In [ ]:
# Helper functions
def make_voxel(origin, index=0, voxel_size=1):
    ox, oy, oz = origin
    vs = voxel_size
    
    x = [ox, ox, ox+vs, ox+vs, ox, ox, ox+vs, ox+vs]
    y = [oy, oy+vs, oy+vs, oy, oy, oy+vs, oy+vs, oy]
    z = [oz, oz, oz, oz, oz+vs, oz+vs, oz+vs, oz+vs]
    
    i = (np.array([7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2]) + index * 8).tolist()
    j = (np.array([3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3]) + index * 8).tolist()
    k = (np.array([0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6]) + index * 8).tolist()
    
    return x, y, z, i, j, k


def get_voxels(voxel_grid):
    x, y, z, i, j, k = [], [], [], [], [], []
    for index, v in enumerate(voxel_grid.get_voxels()):
        voxel = make_voxel(origin=v.grid_index, index=index)
        x.extend(voxel[0])
        y.extend(voxel[1])
        z.extend(voxel[2])
        i.extend(voxel[3])
        j.extend(voxel[4])
        k.extend(voxel[5])
        
    return x, y, z, i, j, k

# Make voxel grid from point cloud and transform to mesh for visualization
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.005)
x, y, z, i, j, k = get_voxels(voxel_grid)

# Use Open3D to remove duplicate and faulty vertices and triangles
vertices = np.vstack([x, y, z]).T
triangles = np.vstack([i, j, k]).T

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)

mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()

mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()

# Convert to Numpy for visualization
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0]
y = vertices[:, 1]
z = vertices[:, 2]
i = triangles[:, 0]
j = triangles[:, 1]
k = triangles[:, 2]

voxel_grid = go.Mesh3d(x=x,
                       y=y,
                       z=z,
                       i=i,
                       j=j,
                       k=k,
                       hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                       hoverlabel=dict(bgcolor='white'),
                       colorscale='magma',
                       intensity=z,
                       flatshading=True,
                       showscale=False)

In [ ]:
# Make figure
fig = make_subplots(rows=1,
                    cols=2,
                    column_widths=[0.5, 0.5],
                    horizontal_spacing=0,
                    vertical_spacing=0,
                    specs=[[dict(type='Mesh3d'), dict(type='Mesh3d')]])

fig.add_trace(point_cloud, row=1, col=1)
fig.add_trace(voxel_grid, row=1, col=2)

# Viewpoint
camera1 = dict(eye=dict(x=-1.8, y=1.3, z=1.9),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

camera2 = dict(eye=dict(x=-1.8, y=1.3, z=1.9),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

fig.update_layout(scene1=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data',
                    camera=camera1),
                  scene2=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data',
                    camera=camera2),
                  height=500,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_dragmode='orbit',
                  showlegend=False)

In [ ]:
# Save figure
pio.write_html(fig,
               file="../_includes/figures/pcd_vs_voxel.html",
               full_html=False,
               include_plotlyjs='cdn')

## The grid

In [ ]:
def make_grid(res, pos=[0, 0, 0], color='black', line_width=1):
    lines = []
    for i in range(res + 1):
        for j in range(res + 1):
            lines.append(go.Scatter3d(x=np.array([0, res]) + pos[0],
                                      y=np.array([i, i]) + pos[1],
                                      z=np.array([j, j]) + pos[2],
                                      mode='lines',
                                      marker=dict(color=color),
                                      line=dict(width=line_width),
                                      showlegend=False))

            lines.append(go.Scatter3d(x=np.array([i, i]) + pos[0],
                                      y=np.array([0, res]) + pos[1],
                                      z=np.array([j, j]) + pos[2],
                                      mode='lines',
                                      marker=dict(color=color),
                                      line=dict(width=line_width),
                                      showlegend=False))

            lines.append(go.Scatter3d(x=np.array([i, i]) + pos[0],
                                      y=np.array([j, j]) + pos[1],
                                      z=np.array([0, res]) + pos[2],
                                      mode='lines',
                                      marker=dict(color=color),
                                      line=dict(width=line_width),
                                      showlegend=False))
    return lines

fig = go.Figure(make_grid(res=18))

# Make voxel grid from point cloud and transform to mesh for visualization
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.01)
x, y, z, i, j, k = get_voxels(voxel_grid)

# Use Open3D to remove duplicate and faulty vertices and triangles
vertices = np.vstack([x, y, z]).T
triangles = np.vstack([i, j, k]).T

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(triangles)

mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()

mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()

# Convert to Numpy for visualization
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
x = vertices[:, 0] + 1
y = vertices[:, 1] + 1
z = vertices[:, 2] + 1
i = triangles[:, 0]
j = triangles[:, 1]
k = triangles[:, 2]

fig.add_trace(go.Mesh3d(x=x,
                        y=y,
                        z=z,
                        i=i,
                        j=j,
                        k=k,
                        hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                        hoverlabel=dict(bgcolor='white'),
                        colorscale='magma',
                        intensity=z,
                        flatshading=True,
                        showscale=False))

# Viewpoint
camera = dict(eye=dict(x=0, y=0, z=2),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0),
              projection=dict(type="orthographic"))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit")

In [ ]:
# Save figure
pio.write_html(fig,
               file="../_includes/figures/voxel_grid.html",
               full_html=False,
               include_plotlyjs='cdn')

## 3D Convolutions

In [ ]:
def conv_3d(index, res, colors, weights):
    # This is not a correct convolution because edge cases are ignored!
    mask = []
    indices = np.arange(-13, 14)
    for i in indices:
        # zero padding
        if i >= 0 and i < res**3:
            mask.append(colors[index + i])
        else:
            mask.append(0)
    return sum(np.array(mask) * np.array(weights))


data = make_grid(res=7, line_width=2) + make_grid(res=3, pos=[4, 4, 4], color='red', line_width=3)

# Input (voxel grid)
grid = np.array(np.meshgrid(np.arange(5), np.arange(5), np.arange(5))).T.reshape(-1, 3)
colors = []
for index, v in enumerate(grid):
    voxel = make_voxel(origin=[v[0], v[1], v[2]], index=0)
    c = np.random.randint(255)
    colors.append(c)
    data.append(go.Mesh3d(x=np.array(voxel[0]) + 1,
                          y=np.array(voxel[1]) + 1,
                          z=np.array(voxel[2]) + 1,
                          i=voxel[3],
                          j=voxel[4],
                          k=voxel[5],
                          hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                          hoverlabel=dict(bgcolor='white'),
                          color=f"rgb{c, 204, 150}",
                          flatshading=True,
                          showscale=False))

# Kernel (weights)
grid = np.array(np.meshgrid(np.arange(3), np.arange(3), np.arange(3))).T.reshape(-1, 3)
weights = []
for index, v in enumerate(grid):
    voxel = make_voxel(origin=[v[0], v[1], v[2]], index=0)
    w = np.random.randint(255)
    weights.append(w * 0.001)
    data.append(go.Mesh3d(x=np.array(voxel[0]) + 4,
                          y=np.array(voxel[1]) + 4,
                          z=np.array(voxel[2]) + 4,
                          i=voxel[3],
                          j=voxel[4],
                          k=voxel[5],
                          hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                          hoverlabel=dict(bgcolor='white'),
                          color=f"rgba{w, w, w, 0.5}",
                          flatshading=True,
                          showscale=False))
    
# Output (feature map)
grid = np.array(np.meshgrid(np.arange(4), np.arange(4), np.arange(4))).T.reshape(-1, 3)
for index, v in enumerate(grid):
    voxel = make_voxel(origin=[v[0], v[1], v[2]], index=0)
    o = conv_3d(index, 5, colors, weights)
    data.append(go.Mesh3d(x=np.array(voxel[0]) + 10,
                          y=np.array(voxel[1]) + 1.5,
                          z=np.array(voxel[2]) + 1.5,
                          i=voxel[3],
                          j=voxel[4],
                          k=voxel[5],
                          hovertemplate="<b>Voxel</b><br>x: %{x}<br>y: %{y}<br>z: %{z}<extra></extra>",
                          hoverlabel=dict(bgcolor='white'),
                          color=f"rgb{255, o, 146}",
                          flatshading=True,
                          showscale=False))

fig = go.Figure(data)

# Viewpoint
camera = dict(eye=dict(x=1.25, y=1.25, z=1.25),
              up=dict(x=0, y=1, z=0),
              center=dict(x=0, y=0, z=0))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera,
                  scene_dragmode="orbit")

In [ ]:
# Save figure
pio.write_html(fig,
               file="../_includes/figures/3d_conv.html",
               full_html=False,
               include_plotlyjs='cdn')